In [73]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('labeled_trip_data.csv')

In [23]:
df['start_timestamp'] = pd.to_datetime(df['start_timestamp'])
df['end_timestamp'] = pd.to_datetime(df['end_timestamp'])
df['next_trip_start_timestamp'] = pd.to_datetime(df['next_trip_start_timestamp'])


df['trip_duration_mins'] = (df['end_timestamp'] - df['start_timestamp']).astype('timedelta64[m]')

df['dwell_time_mins'] = (df['next_trip_start_timestamp'] - df['end_timestamp'] ).astype('timedelta64[m]')

In [16]:
df.head()

,Unnamed: 0,seq,Lat_x,Lon_x,vin,cluster_labels,Lat_y,Lon_y,start_timestamp,end_timestamp,arrival_hour,arrival_date,arrival_day,trip_time,next_trip_start_timestamp,next_trip_seq,dwell_time,seq_diff,trip_duration_mins
0,0,561,40.148659,-82.903959,008beb5671fb49a92dc0ef9de96979bf,0,40.148659,-82.903959,2020-01-09 19:29:29,2020-01-09 19:53:56,19.0,2020-01-09,Thursday,0 days 00:24:27,2020-01-10 17:54:30,562.0,0 days 22:00:34,1.0,24.0
1,3,570,40.148641,-82.903938,008beb5671fb49a92dc0ef9de96979bf,0,40.148641,-82.903938,2020-01-11 11:03:33,2020-01-11 11:05:10,11.0,2020-01-11,Saturday,0 days 00:01:37,2020-01-11 12:05:24,571.0,0 days 01:00:14,1.0,1.0
2,4,573,40.148644,-82.904189,008beb5671fb49a92dc0ef9de96979bf,0,40.148644,-82.904189,2020-01-11 13:16:06,2020-01-11 13:16:20,13.0,2020-01-11,Saturday,0 days 00:00:14,2020-01-11 16:37:27,574.0,0 days 03:21:07,1.0,0.0
3,5,576,40.148642,-82.903986,008beb5671fb49a92dc0ef9de96979bf,0,40.148642,-82.903986,2020-01-11 17:17:31,2020-01-11 17:39:49,17.0,2020-01-11,Saturday,0 days 00:22:18,2020-01-11 17:43:30,577.0,0 days 00:03:41,1.0,22.0
4,8,591,40.148097,-82.977143,008beb5671fb49a92dc0ef9de96979bf,1,40.148097,-82.977143,2020-01-15 09:11:56,2020-01-15 09:22:13,9.0,2020-01-15,Wednesday,0 days 00:10:17,2020-01-15 10:35:44,592.0,0 days 01:13:31,1.0,10.0


In [63]:
average_trip_duration_cluster = df.groupby('cluster_labels', as_index=False)['trip_duration_mins'].mean()
average_trip_duration_vin = df.groupby('vin', as_index=False)['trip_duration_mins'].mean()

In [69]:
absolute_visits = df.groupby('cluster_labels', as_index=False)['vin'].count()
unique_visits = df.groupby('cluster_labels', as_index=False).agg({'vin': pd.Series.nunique})

unique_visits.head()

,cluster_labels,vin
0,0,1
1,1,18
2,2,13
3,3,8
4,4,45


In [50]:
avg_daily_base = df.groupby(['cluster_labels', 'arrival_day'], as_index=False)['start_timestamp'].count()

avg_daily = avg_daily_base.groupby('cluster_labels', as_index=False)['start_timestamp'].mean()

In [47]:
average_dwell_cluster = df.groupby('cluster_labels', as_index=False)['dwell_time_mins'].mean()


In [77]:
avg_daily_merged = avg_daily.merge(average_dwell_cluster)
avg_daily_merged = avg_daily_merged.rename(columns = {'start_timestamp': 'avg_daily_count'})
avg_trip_duration_merged = avg_daily_merged.merge(average_trip_duration_cluster)
avg_trip_counts_merged = avg_trip_duration_merged.merge(absolute_visits).rename(columns = {'vin': 'absolute_counts'})
final_merged = avg_trip_counts_merged.merge(unique_visits).rename(columns = {'vin': 'unique_visits'})

utility_score_cols = [cols for cols in final_merged.columns if cols != 'cluster_labels']

final_merged['utility_score'] = final_merged[utility_score_cols].apply(np.prod, axis=1)

In [78]:
final_merged

,cluster_labels,avg_daily_count,dwell_time_mins,trip_duration_mins,absolute_counts,unique_visits,utility_score
0,0,2.000000,396.000000,11.750000,4,1,3.722400e+04
1,1,3.142857,18.727273,13.090909,22,18,3.051154e+05
2,2,2.857143,314.150000,18.250000,20,13,4.258976e+06
3,3,1.600000,15.000000,13.750000,8,8,2.112000e+04
4,4,13.571429,258.189474,19.610526,95,45,2.937578e+08
...,...,...,...,...,...,...,...
77,77,1.666667,19.900000,23.000000,10,10,7.628333e+04
78,78,2.200000,301.818182,15.454545,11,6,6.772800e+05
79,79,1.142857,483.750000,41.000000,8,2,3.626743e+05
80,80,2.000000,508.125000,3.750000,8,1,3.048750e+04
